<a href="https://colab.research.google.com/github/Preet28/Fake-Review-Detection/blob/main/Improved_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Preet28/Fake-Review-Detection.git

Cloning into 'Fake-Review-Detection'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 65 (delta 33), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (65/65), 927.02 KiB | 2.62 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [15]:
from textblob import TextBlob
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents
import regex as re
import operator
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [7]:
df = pd.read_csv('/content/Fake-Review-Detection/deceptive-opinion.csv')
df.head(3)

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...


In [8]:
df = df.drop(['hotel', 'source', 'polarity'], axis=1)
df.head(2)

,deceptive,text
0,truthful,We stayed for a one night getaway with family ...
1,truthful,Triple A rate with upgrade to view room was le...


In [11]:
df['text'] = df['text'].map(lambda x: x.lower())
df['text'].head(3)

0    we stayed for a one night getaway with family ...
1    triple a rate with upgrade to view room was le...
2    this comes a little late as i'm finally catchi...
Name: text, dtype: object

In [12]:
df.head(4)

,deceptive,text
0,truthful,we stayed for a one night getaway with family ...
1,truthful,triple a rate with upgrade to view room was le...
2,truthful,this comes a little late as i'm finally catchi...
3,truthful,the omni chicago really delivers on all fronts...


In [16]:
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents
nltk.download('stopwords')

stop = stopwords.words('english')
df['review_without_stopwords'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
df

,deceptive,text,review_without_stopwords
0,truthful,we stayed for a one night getaway with family ...,stayed one night getaway family thursday. trip...
1,truthful,triple a rate with upgrade to view room was le...,triple rate upgrade view room less $200 also i...
2,truthful,this comes a little late as i'm finally catchi...,comes little late i'm finally catching reviews...
3,truthful,the omni chicago really delivers on all fronts...,"omni chicago really delivers fronts, spaciousn..."
4,truthful,i asked for a high floor away from the elevato...,asked high floor away elevator got. room pleas...
...,...,...,...
1595,deceptive,problems started when i booked the intercontin...,problems started booked intercontinental chica...
1596,deceptive,the amalfi hotel has a beautiful website and i...,amalfi hotel beautiful website interior decora...
1597,deceptive,the intercontinental chicago magnificent mile ...,intercontinental chicago magnificent mile outs...
1598,deceptive,"the palmer house hilton, while it looks good i...","palmer house hilton, looks good pictures, outs..."


In [18]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [20]:
def pos(review_without_stopwords):
    return TextBlob(review_without_stopwords).tags

os = df.review_without_stopwords.apply(pos)
os1 = pd.DataFrame(os)

In [21]:
os1

,review_without_stopwords
0,"[(stayed, JJ), (one, CD), (night, NN), (getawa..."
1,"[(triple, JJ), (rate, NN), (upgrade, JJ), (vie..."
2,"[(comes, VBZ), (little, JJ), (late, JJ), (i, N..."
3,"[(omni, NNS), (chicago, VBP), (really, RB), (d..."
4,"[(asked, VBN), (high, JJ), (floor, NN), (away,..."
...,...
1595,"[(problems, NNS), (started, VBD), (booked, VBD..."
1596,"[(amalfi, JJ), (hotel, NN), (beautiful, JJ), (..."
1597,"[(intercontinental, JJ), (chicago, NN), (magni..."
1598,"[(palmer, NN), (house, NN), (hilton, NN), (loo..."


In [22]:
os1['pos'] = os1['review_without_stopwords'].map(lambda x:" ".join(["".join(x) for x in x ]) )

In [23]:
df = pd.merge(df, os1, right_index=True, left_index = True)
df.head()

,deceptive,text,review_without_stopwords_x,review_without_stopwords_y,pos
0,truthful,we stayed for a one night getaway with family ...,stayed one night getaway family thursday. trip...,"[(stayed, JJ), (one, CD), (night, NN), (getawa...",stayedJJ oneCD nightNN getawayNN familyNN thur...
1,truthful,triple a rate with upgrade to view room was le...,triple rate upgrade view room less $200 also i...,"[(triple, JJ), (rate, NN), (upgrade, JJ), (vie...",tripleJJ rateNN upgradeJJ viewNN roomNN lessJJ...
2,truthful,this comes a little late as i'm finally catchi...,comes little late i'm finally catching reviews...,"[(comes, VBZ), (little, JJ), (late, JJ), (i, N...",comesVBZ littleJJ lateJJ iNN 'mVBP finallyRB c...
3,truthful,the omni chicago really delivers on all fronts...,"omni chicago really delivers fronts, spaciousn...","[(omni, NNS), (chicago, VBP), (really, RB), (d...",omniNNS chicagoVBP reallyRB deliversNNS fronts...
4,truthful,i asked for a high floor away from the elevato...,asked high floor away elevator got. room pleas...,"[(asked, VBN), (high, JJ), (floor, NN), (away,...",askedVBN highJJ floorNN awayRB elevatorJJ gotV...


In [26]:
review_train, review_test, label_train, label_test = train_test_split(df['pos'], df['deceptive'], test_size=0.2,random_state=13)

In [27]:
tf_vect = TfidfVectorizer(lowercase = True, use_idf=True, smooth_idf=True, sublinear_tf=False)

X_train_tf = tf_vect.fit_transform(review_train)
X_test_tf = tf_vect.transform(review_test)

In [28]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='linear'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    return grid_search.best_params_

svc_param_selection(X_train_tf,label_train,5)

{'C': 1, 'gamma': 0.001}

In [29]:
{'C': 10, 'gamma': 0.001}

{'C': 10, 'gamma': 0.001}

In [43]:
clf = svm.SVC(C=10,gamma=0.001,kernel='linear')
clf.fit(X_train_tf,label_train)
pred = clf.predict(X_test_tf)

For Random State = 10

In [40]:
review_train, review_test, label_train, label_test = train_test_split(df['pos'],df['deceptive'], test_size=0.2,random_state=10)
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)
print(metrics.accuracy_score(label_test, pred))
print (confusion_matrix(label_test, pred))
print (classification_report(label_test, pred))

0.971875
[[155   5]
 [  4 156]]
              precision    recall  f1-score   support

   deceptive       0.97      0.97      0.97       160
    truthful       0.97      0.97      0.97       160

    accuracy                           0.97       320
   macro avg       0.97      0.97      0.97       320
weighted avg       0.97      0.97      0.97       320



Random State - 42

In [41]:
review_train, review_test, label_train, label_test = train_test_split(df['pos'],df['deceptive'], test_size=0.2,random_state=42)
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)
print(metrics.accuracy_score(label_test, pred))
print (confusion_matrix(label_test, pred))
print (classification_report(label_test, pred))

0.978125
[[149   3]
 [  4 164]]
              precision    recall  f1-score   support

   deceptive       0.97      0.98      0.98       152
    truthful       0.98      0.98      0.98       168

    accuracy                           0.98       320
   macro avg       0.98      0.98      0.98       320
weighted avg       0.98      0.98      0.98       320

